In [19]:
import pandas as pd

unemployment = pd.read_csv("Unemployment.csv")
unemployment

,FIPS_Code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
0,0,US,United States,NaN,NaN,NaN,"142,601,576","136,904,853","5,696,723",4.0,...,"163,140,305","157,154,185","5,986,120",3.7,"160,611,064","147,677,360","12,933,704",8.1,"65,712",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,147,173","2,047,731","99,442",4.6,...,"2,237,278","2,169,395","67,883",3.0,"2,230,118","2,099,062","131,056",5.9,"51,771",100.0
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,861","20,971",890,4.1,...,"26,281","25,561",720,2.7,"25,838","24,576","1,262",4.9,"58,233",112.5
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,979","67,370","2,609",3.7,...,"97,424","94,713","2,711",2.8,"96,763","91,338","5,425",5.6,"59,871",115.6
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,449","10,812",637,5.6,...,"8,502","8,177",325,3.8,"8,587","7,982",605,7.0,"35,972",69.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270,72145,PR,"Vega Baja Municipio, PR",1.0,1.0,1.0,"19,805","17,607","2,198",11.1,...,"13,172","11,910","1,262",9.6,NaN,NaN,NaN,NaN,NaN,NaN
3271,72147,PR,"Vieques Municipio, PR",7.0,12.0,0.0,"2,441","2,120",321,13.2,...,"2,616","2,433",183,7.0,NaN,NaN,NaN,NaN,NaN,NaN
3272,72149,PR,"Villalba Municipio, PR",2.0,2.0,1.0,"7,636","6,648",988,12.9,...,"7,492","6,307","1,185",15.8,NaN,NaN,NaN,NaN,NaN,NaN
3273,72151,PR,"Yabucoa Municipio, PR",1.0,1.0,1.0,"10,613","9,005","1,608",15.2,...,"8,840","7,685","1,155",13.1,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
ca_unemployment = unemployment[unemployment.State=="CA"]
ca_unemployment

,FIPS_Code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
194,6000,CA,California,NaN,NaN,NaN,"16,837,536","16,011,619","825,917",4.9,...,"19,353,743","18,550,526","803,217",4.2,"18,821,167","16,913,078","1,908,089",10.1,"80,423",100.0
195,6001,CA,"Alameda County, CA",1.0,1.0,1.0,"768,654","741,264","27,390",3.6,...,"841,075","815,889","25,186",3.0,"813,807","742,411","71,396",8.8,"107,589",133.8
196,6003,CA,"Alpine County, CA",8.0,4.0,0.0,555,520,35,6.3,...,541,512,29,5.4,517,459,58,11.2,"58,112",72.3
197,6005,CA,"Amador County, CA",6.0,4.0,0.0,"15,260","14,477",783,5.1,...,"14,801","14,217",584,3.9,"14,424","13,107","1,317",9.1,"62,640",77.9
198,6007,CA,"Butte County, CA",3.0,2.0,1.0,"93,071","87,362","5,709",6.1,...,"97,922","92,858","5,064",5.2,"92,604","84,095","8,509",9.2,"58,394",72.6
199,6009,CA,"Calaveras County, CA",6.0,6.0,0.0,"18,133","17,134",999,5.5,...,"21,310","20,485",825,3.9,"21,207","19,588","1,619",7.6,"68,248",84.9
200,6011,CA,"Colusa County, CA",6.0,4.0,0.0,"9,253","8,194","1,059",11.4,...,"10,975","9,549","1,426",13.0,"10,499","8,824","1,675",16.0,"59,048",73.4
201,6013,CA,"Contra Costa County, CA",1.0,1.0,1.0,"500,681","483,153","17,528",3.5,...,"559,710","542,121","17,589",3.1,"541,256","493,236","48,020",8.9,"106,555",132.5
202,6015,CA,"Del Norte County, CA",7.0,8.0,0.0,"10,228","9,478",750,7.3,...,"9,617","9,053",564,5.9,"9,350","8,466",884,9.5,"48,979",60.9
203,6017,CA,"El Dorado County, CA",1.0,1.0,1.0,"82,122","78,763","3,359",4.1,...,"92,132","88,833","3,299",3.6,"90,752","83,262","7,490",8.3,"86,202",107.2


In [30]:
ca_unemployment['Area_name'] = ca_unemployment['Area_name'].str.replace(', CA', '')
ca_unemployment

/tmp/ipykernel_195/3019267566.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ca_unemployment['Area_name'] = ca_unemployment['Area_name'].str.replace(', CA', '')


,FIPS_Code,State,Area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Civilian_labor_force_2020,Employed_2020,Unemployed_2020,Unemployment_rate_2020,Median_Household_Income_2019,Med_HH_Income_Percent_of_State_Total_2019
194,6000,CA,California,NaN,NaN,NaN,"16,837,536","16,011,619","825,917",4.9,...,"19,353,743","18,550,526","803,217",4.2,"18,821,167","16,913,078","1,908,089",10.1,"80,423",100.0
195,6001,CA,Alameda County,1.0,1.0,1.0,"768,654","741,264","27,390",3.6,...,"841,075","815,889","25,186",3.0,"813,807","742,411","71,396",8.8,"107,589",133.8
196,6003,CA,Alpine County,8.0,4.0,0.0,555,520,35,6.3,...,541,512,29,5.4,517,459,58,11.2,"58,112",72.3
197,6005,CA,Amador County,6.0,4.0,0.0,"15,260","14,477",783,5.1,...,"14,801","14,217",584,3.9,"14,424","13,107","1,317",9.1,"62,640",77.9
198,6007,CA,Butte County,3.0,2.0,1.0,"93,071","87,362","5,709",6.1,...,"97,922","92,858","5,064",5.2,"92,604","84,095","8,509",9.2,"58,394",72.6
199,6009,CA,Calaveras County,6.0,6.0,0.0,"18,133","17,134",999,5.5,...,"21,310","20,485",825,3.9,"21,207","19,588","1,619",7.6,"68,248",84.9
200,6011,CA,Colusa County,6.0,4.0,0.0,"9,253","8,194","1,059",11.4,...,"10,975","9,549","1,426",13.0,"10,499","8,824","1,675",16.0,"59,048",73.4
201,6013,CA,Contra Costa County,1.0,1.0,1.0,"500,681","483,153","17,528",3.5,...,"559,710","542,121","17,589",3.1,"541,256","493,236","48,020",8.9,"106,555",132.5
202,6015,CA,Del Norte County,7.0,8.0,0.0,"10,228","9,478",750,7.3,...,"9,617","9,053",564,5.9,"9,350","8,466",884,9.5,"48,979",60.9
203,6017,CA,El Dorado County,1.0,1.0,1.0,"82,122","78,763","3,359",4.1,...,"92,132","88,833","3,299",3.6,"90,752","83,262","7,490",8.3,"86,202",107.2


In [27]:
education = pd.read_csv("Education.csv")
education

,FIPS Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,"Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2015-19","High school diploma only, 2015-19","Some college or associate's degree, 2015-19","Bachelor's degree or higher, 2015-19","Percent of adults with less than a high school diploma, 2015-19","Percent of adults with a high school diploma only, 2015-19","Percent of adults completing some college or associate's degree, 2015-19","Percent of adults with a bachelor's degree or higher, 2015-19"
0,0,US,United States,NaN,NaN,NaN,NaN,"52,373,312","34,158,051","11,650,730",...,27.4,24.4,"26,472,261","59,472,748","63,756,905","70,920,162",12.0,27.0,28.9,32.1
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,"1,062,306","468,269","136,287",...,25.9,19.0,"458,922","1,022,839","993,344","845,772",13.8,30.8,29.9,25.5
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,"6,611","3,757",933,...,26.9,18.0,"4,291","12,551","10,596","9,929",11.5,33.6,28.4,26.6
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,"18,726","8,426","2,334",...,29.3,23.1,"13,893","41,797","47,274","48,148",9.2,27.7,31.3,31.9
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,"8,120","2,242",581,...,21.3,10.9,"4,812","6,396","4,676","2,080",26.8,35.6,26.0,11.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278,72145,PR,Vega Baja Municipio,1.0,1.0,1.0,1.0,NaN,NaN,NaN,...,21.4,14.4,"10,494","9,681","8,905","7,834",28.4,26.2,24.1,21.2
3279,72147,PR,Vieques Municipio,7.0,12.0,7.0,12.0,NaN,NaN,NaN,...,13.7,10.1,"1,792","2,440",875,"1,121",28.8,39.2,14.0,18.0
3280,72149,PR,Villalba Municipio,2.0,2.0,2.0,2.0,NaN,NaN,NaN,...,18.0,12.8,"3,330","5,809","2,987","3,015",22.0,38.4,19.7,19.9
3281,72151,PR,Yabucoa Municipio,1.0,1.0,1.0,1.0,NaN,NaN,NaN,...,18.3,12.1,"6,927","6,132","6,494","4,293",29.0,25.7,27.2,18.0


In [28]:
ca_education = education[education.State=="CA"]
ca_education

,FIPS Code,State,Area name,2003 Rural-urban Continuum Code,2003 Urban Influence Code,2013 Rural-urban Continuum Code,2013 Urban Influence Code,"Less than a high school diploma, 1970","High school diploma only, 1970","Some college (1-3 years), 1970",...,"Percent of adults completing some college or associate's degree, 2000","Percent of adults with a bachelor's degree or higher, 2000","Less than a high school diploma, 2015-19","High school diploma only, 2015-19","Some college or associate's degree, 2015-19","Bachelor's degree or higher, 2015-19","Percent of adults with less than a high school diploma, 2015-19","Percent of adults with a high school diploma only, 2015-19","Percent of adults completing some college or associate's degree, 2015-19","Percent of adults with a bachelor's degree or higher, 2015-19"
198,6000,CA,California,NaN,NaN,NaN,NaN,"4,062,367","3,566,578","1,784,772",...,30.0,26.6,"4,418,675","5,423,462","7,648,680","8,980,726",16.7,20.5,28.9,33.9
199,6001,CA,Alameda County,1.0,1.0,1.0,1.0,"217,589","197,371","87,675",...,28.4,34.9,"135,524","205,980","275,053","556,105",11.6,17.6,23.5,47.4
200,6003,CA,Alpine County,8.0,4.0,8.0,4.0,101,98,57,...,34.1,28.2,68,216,220,266,8.8,28.1,28.6,34.5
201,6005,CA,Amador County,6.0,4.0,6.0,4.0,"2,911","2,314","1,214",...,37.1,16.6,"3,024","8,322","13,112","5,837",10.0,27.5,43.3,19.3
202,6007,CA,Butte County,3.0,2.0,3.0,2.0,"24,482","17,743","8,511",...,36.1,21.8,"15,864","33,186","58,310","40,149",10.8,22.5,39.5,27.2
203,6009,CA,Calaveras County,6.0,6.0,6.0,6.0,"4,164","2,721","1,127",...,40.0,17.1,"3,424","9,759","15,281","6,384",9.8,28.0,43.9,18.3
204,6011,CA,Colusa County,6.0,4.0,6.0,4.0,"3,104","2,188","1,011",...,29.2,10.6,"3,932","3,586","4,107","2,052",28.7,26.2,30.0,15.0
205,6013,CA,Contra Costa County,1.0,1.0,1.0,1.0,"96,353","101,602","50,423",...,32.1,35.0,"82,921","135,825","235,034","333,398",10.5,17.3,29.9,42.4
206,6015,CA,Del Norte County,7.0,8.0,7.0,8.0,"3,970","2,592",801,...,33.2,11.0,"3,918","6,095","6,724","2,907",19.9,31.0,34.2,14.8
207,6017,CA,El Dorado County,1.0,1.0,1.0,1.0,"9,251","9,356","4,194",...,40.3,26.5,"9,029","29,137","51,867","46,995",6.6,21.3,37.9,34.3


In [22]:
monthly_transportation = pd.read_csv("Monthly_Transportation_Statistics.csv")
monthly_transportation

,Index,Date,Air Safety - General Aviation Fatalities,Highway Fatalities Per 100 Million Vehicle Miles Traveled,Highway Fatalities,U.S. Airline Traffic - Total - Seasonally Adjusted,U.S. Airline Traffic - International - Seasonally Adjusted,U.S. Airline Traffic - Domestic - Seasonally Adjusted,Transit Ridership - Other Transit Modes - Adjusted,Transit Ridership - Fixed Route Bus - Adjusted,...,Heavy truck sales SAAR (millions),U.S. Airline Traffic - Total - Non Seasonally Adjusted,Light truck sales SAAR (millions),U.S. Airline Traffic - International - Non Seasonally Adjusted,Auto sales SAAR (millions),U.S. Airline Traffic - Domestic - Non Seasonally Adjusted,Transborder - Total North American Freight,Transborder - U.S. - Mexico Freight,U.S. marketing air carriers on-time performance (percent),Transborder - U.S. - Canada Freight
0,0,01/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,02/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,03/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,04/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,05/01/1947 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,894,07/01/2021 12:00:00 AM,52.0,NaN,NaN,65270000.0,5640000.0,59280000.0,9753916.0,205637880.0,...,440000.0,73720000.0,11202000.0,7030000.0,3479000.0,66690000.0,1.112701e+11,5.556935e+10,0.7,5.570078e+10
895,895,08/01/2021 12:00:00 AM,25.0,NaN,NaN,NaN,5610000.0,57330000.0,9794587.0,215644069.0,...,417000.0,66930000.0,9997000.0,6360000.0,2988000.0,60570000.0,1.130751e+11,5.626947e+10,0.7,5.680559e+10
896,896,09/01/2021 12:00:00 AM,20.0,NaN,NaN,NaN,NaN,NaN,9426917.0,233718787.0,...,405000.0,NaN,9544000.0,NaN,2614000.0,NaN,1.092200e+11,5.483225e+10,NaN,5.438774e+10
897,897,10/01/2021 12:00:00 AM,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
